In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# build_model_from_ttl_minio.py — dérive le dataset tabulaire depuis training_room_semantic_full.ttl
# Sorties locales: ~/DTE/jne_project/model/{YYYY-MM}/ (ttl + csv + parquet + manifest)
# Upload: s3://model/jne_project/model/{YYYY-MM}/...

import os, sys, json, argparse
from pathlib import Path
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF

BRICK = Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = Namespace("https://brickschema.org/schema/BrickFrame#")
EX    = Namespace("http://example.org/training#")

# ---------- MinIO ----------
def s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client("s3", endpoint_url=endpoint,
        aws_access_key_id=access, aws_secret_access_key=secret,
        use_ssl=bool(secure), verify=bool(secure),
        region_name="us-east-1", config=Config(signature_version="s3v4"))

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload(s3, bucket, p:Path, key:str): s3.upload_file(str(p), bucket, key)

# ---------- TTL → mapping (csv_uri, column) + BIM ----------
def parse_ttl(ttl_path: Path):
    g = Graph().parse(str(ttl_path), format="turtle")
    mapping = []  # [{sensor, csv_uri, column}]
    for s, _, o in g.triples((None, BF.hasTimeseriesId, None)):
        tsid = str(o)
        if "::" in tsid:
            uri, col = tsid.split("::", 1)
            mapping.append({"sensor": str(s), "csv_uri": uri, "column": col})
    # BIM: toutes les propriétés EX:* de la pièce
    bim = {}
    room = EX["Room_101"]
    if (room, RDF.type, None) not in g:
        rooms = list(g.subjects(RDF.type, BRICK.Room))
        if rooms: room = rooms[0]
    for p, o in g.predicate_objects(room):
        sp = str(p)
        if sp.startswith(str(EX)):
            key = "BIM_" + sp.split("#")[-1]
            val = getattr(o, "toPython", lambda: o)()
            bim[key] = str(val)
    return mapping, bim

# ---------- Chargement et fusion ----------
def build_table(mapping, refined_root="~/DTE/jne_project/refined"):
    cache = {}
    dfs = []
    for m in mapping:
        local = Path(m["csv_uri"].replace("minio://refined", refined_root)).expanduser().resolve()
        if not local.exists():
            print(f"WARN: fichier absent {local}")
            continue
        if local not in cache:
            cache[local] = pd.read_csv(local)
        df = cache[local]
        col = m["column"]
        if "ts" not in df.columns or col not in df.columns:
            print(f"WARN: colonnes manquantes dans {local} ({col})")
            continue
        # nom de colonne lisible = suffixe de l'URI du capteur
        cname = m["sensor"].split("#")[-1]
        dfs.append(df[["ts", col]].rename(columns={col: cname}))
    if not dfs:
        return None
    out = dfs[0]
    for d in dfs[1:]:
        out = out.merge(d, on="ts", how="outer")
    # tri + ISO
    out = out.sort_values("ts")
    return out

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")
    ap.add_argument("--ttl_full", type=str, default="~/DTE/jne_project/semantic/{month}/training_room_semantic_full.ttl")
    ap.add_argument("--model_base", type=str, default="~/DTE/jne_project/model")
    # MinIO model
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="model")
    ap.add_argument("--prefix",   type=str, default="jne_project/model")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    month = args.month
    ttl_path = Path(args.ttl_full.format(month=month)).expanduser().resolve()

    model_dir = Path(args.model_base).expanduser().resolve() / month
    meta_dir  = Path(args.model_base).expanduser().resolve() / "meta" / month
    model_dir.mkdir(parents=True, exist_ok=True)
    meta_dir.mkdir(parents=True, exist_ok=True)

    mapping, bim = parse_ttl(ttl_path)
    df = build_table(mapping)
    if df is None:
        print("ERREUR: aucune série chargée depuis le TTL"); sys.exit(1)

    # Ajouter BIM (constantes)
    for k, v in bim.items():
        df[k] = v

    # Sauvegardes locales
    out_ttl_copy = model_dir / "training_room_model.ttl"
    out_csv      = model_dir / "training_room_model_dataset.csv"
    out_parquet  = model_dir / "training_room_model_dataset.parquet"

    # copier le TTL complet tel quel
    out_ttl_copy.write_text(Path(ttl_path).read_text(encoding="utf-8"), encoding="utf-8")
    df.to_csv(out_csv, index=False)
    df.to_parquet(out_parquet, index=False)

    manifest = {
        "version":"1.0",
        "month": month,
        "ttl_input": str(ttl_path),
        "outputs": {"ttl": str(out_ttl_copy), "csv": str(out_csv), "parquet": str(out_parquet)},
        "rows": int(len(df)), "columns": list(df.columns),
        "mapping_count": len(mapping),
        "bim_keys": list(bim.keys())
    }
    man_path = meta_dir / "model_manifest.json"
    man_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    # Upload MinIO
    if not args.no_upload:
        try:
            s3 = s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            s3_upload(s3, args.bucket, out_ttl_copy, f"{root}/{month}/training_room_model.ttl")
            s3_upload(s3, args.bucket, out_csv,      f"{root}/{month}/training_room_model_dataset.csv")
            s3_upload(s3, args.bucket, out_parquet,  f"{root}/{month}/training_room_model_dataset.parquet")
            s3_upload(s3, args.bucket, man_path,     f"{root}/meta/{month}/model_manifest.json")
            print("minio:", f"s3://{args.bucket}/{root}/{month}/training_room_model_dataset.parquet")
        except Exception as e:
            print("ERREUR MinIO:", e); sys.exit(3)

    print("OK:", out_ttl_copy, out_csv, out_parquet, man_path)

if __name__ == "__main__":
    main()


WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
minio: s3://model/jne_project/model/2025-03/training_room_model_dataset.parquet
OK: /home/amina/DTE/jne_project/model/2025-03/training_room_model.ttl /home/amina/DTE/jne_project/model/2025-03/training_room_model_dataset.csv /home/amina/DTE/jne_project/model/2025-03/training_room_model_dataset.parquet /home/amina/DTE/jne_project/model/meta/2025-03/model_manifest.json


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# build_model_from_ttl_minio.py — dérive le dataset tabulaire depuis training_room_semantic_full.ttl
# Sorties locales: ~/DTE/jne_project/model/{YYYY-MM}/ (ttl + csv + parquet + manifest)
# Upload: s3://model/jne_project/model/{YYYY-MM}/...

import os, sys, json, argparse
from pathlib import Path
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF

BRICK = Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = Namespace("https://brickschema.org/schema/BrickFrame#")
EX    = Namespace("http://example.org/training#")

# ---------- MinIO ----------
def s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client("s3", endpoint_url=endpoint,
        aws_access_key_id=access, aws_secret_access_key=secret,
        use_ssl=bool(secure), verify=bool(secure),
        region_name="us-east-1", config=Config(signature_version="s3v4"))

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload(s3, bucket, p:Path, key:str): s3.upload_file(str(p), bucket, key)

# ---------- TTL → mapping (csv_uri, column) + BIM ----------
def parse_ttl(ttl_path: Path):
    g = Graph().parse(str(ttl_path), format="turtle")
    mapping = []  # [{sensor, csv_uri, column}]
    for s, _, o in g.triples((None, BF.hasTimeseriesId, None)):
        tsid = str(o)
        if "::" in tsid:
            uri, col = tsid.split("::", 1)
            mapping.append({"sensor": str(s), "csv_uri": uri, "column": col})
    # BIM: toutes les propriétés EX:* de la pièce
    bim = {}
    room = EX["Room_101"]
    if (room, RDF.type, None) not in g:
        rooms = list(g.subjects(RDF.type, BRICK.Room))
        if rooms: room = rooms[0]
    for p, o in g.predicate_objects(room):
        sp = str(p)
        if sp.startswith(str(EX)):
            key = "BIM_" + sp.split("#")[-1]
            val = getattr(o, "toPython", lambda: o)()
            bim[key] = str(val)
    return mapping, bim

# ---------- Chargement et fusion ----------
def build_table(mapping, refined_root="~/DTE/jne_project/refined"):
    cache = {}
    dfs = []
    for m in mapping:
        local = Path(m["csv_uri"].replace("minio://refined", refined_root)).expanduser().resolve()
        if not local.exists():
            print(f"WARN: fichier absent {local}")
            continue
        if local not in cache:
            cache[local] = pd.read_csv(local)
        df = cache[local]
        col = m["column"]
        if "ts" not in df.columns or col not in df.columns:
            print(f"WARN: colonnes manquantes dans {local} ({col})")
            continue
        # nom de colonne lisible = suffixe de l'URI du capteur
        cname = m["sensor"].split("#")[-1]
        dfs.append(df[["ts", col]].rename(columns={col: cname}))
    if not dfs:
        return None
    out = dfs[0]
    for d in dfs[1:]:
        out = out.merge(d, on="ts", how="outer")
    # tri + ISO
    out = out.sort_values("ts")
    return out

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")
    ap.add_argument("--ttl_full", type=str, default="~/DTE/jne_project/semantic/{month}/training_room_semantic_full.ttl")
    ap.add_argument("--model_base", type=str, default="~/DTE/jne_project/model")
    # MinIO model
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="model")
    ap.add_argument("--prefix",   type=str, default="jne_project/model")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    month = args.month
    ttl_path = Path(args.ttl_full.format(month=month)).expanduser().resolve()

    model_dir = Path(args.model_base).expanduser().resolve() / month
    meta_dir  = Path(args.model_base).expanduser().resolve() / "meta" / month
    model_dir.mkdir(parents=True, exist_ok=True)
    meta_dir.mkdir(parents=True, exist_ok=True)

    mapping, bim = parse_ttl(ttl_path)
    df = build_table(mapping)
    if df is None:
        print("ERREUR: aucune série chargée depuis le TTL"); sys.exit(1)

    # Ajouter BIM (constantes)
    for k, v in bim.items():
        df[k] = v

    # Sauvegardes locales
    out_ttl_copy = model_dir / "training_room_model.ttl"
    out_csv      = model_dir / "training_room_model_dataset.csv"
    out_parquet  = model_dir / "training_room_model_dataset.parquet"

    # copier le TTL complet tel quel
    out_ttl_copy.write_text(Path(ttl_path).read_text(encoding="utf-8"), encoding="utf-8")
    df.to_csv(out_csv, index=False)
    df.to_parquet(out_parquet, index=False)

    manifest = {
        "version":"1.0",
        "month": month,
        "ttl_input": str(ttl_path),
        "outputs": {"ttl": str(out_ttl_copy), "csv": str(out_csv), "parquet": str(out_parquet)},
        "rows": int(len(df)), "columns": list(df.columns),
        "mapping_count": len(mapping),
        "bim_keys": list(bim.keys())
    }
    man_path = meta_dir / "model_manifest.json"
    man_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    # Upload MinIO
    if not args.no_upload:
        try:
            s3 = s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            s3_upload(s3, args.bucket, out_ttl_copy, f"{root}/{month}/training_room_model.ttl")
            s3_upload(s3, args.bucket, out_csv,      f"{root}/{month}/training_room_model_dataset.csv")
            s3_upload(s3, args.bucket, out_parquet,  f"{root}/{month}/training_room_model_dataset.parquet")
            s3_upload(s3, args.bucket, man_path,     f"{root}/meta/{month}/model_manifest.json")
            print("minio:", f"s3://{args.bucket}/{root}/{month}/training_room_model_dataset.parquet")
        except Exception as e:
            print("ERREUR MinIO:", e); sys.exit(3)

    print("OK:", out_ttl_copy, out_csv, out_parquet, man_path)

if __name__ == "__main__":
    main()


WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
WARN: fichier absent /home/amina/DTE/jne_project/refined/sensors/2025-03/zone_101_sensors_refined.csv
minio: s3://model/jne_project/model/2025-03/training_room_model_dataset.parquet
OK: /home/amina/DTE/jne_project/model/2025-03/training_room_model.ttl /home/amina/DTE/jne_project/model/2025-03/training_room_model_dataset.csv /home/amina/DTE/jne_project/model/2025-03/training_room_model_dataset.parquet /home/amina/DTE/jne_project/model/meta/2025-03/model_manifest.json


In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# visualize_semantic_compact.py — Graphe lisible Brick+BIM+Météo+IoT (A4, SVG+PNG)

from pathlib import Path
import re, argparse
import pandas as pd
import rdflib
import pygraphviz as pgv

BRICK = rdflib.Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = rdflib.Namespace("https://brickschema.org/schema/BrickFrame#")
EX    = rdflib.Namespace("http://example.org/training#")

# ---------- utils ----------
def short(u):
    s = str(u)
    return s.split("#")[-1] if "#" in s else s.rsplit("/",1)[-1]

def clean_label(s):
    return re.sub(r"_+", " ", str(s)).strip()

def last_and_stats(df, col):
    if col not in df.columns:
        return None, None, None, None
    s = df[col]
    if s.dtype == bool:
        s = s.astype(int)
    cat_map = {"low":0.25,"normal":0.5,"med":0.75,"medium":0.75,"high":1.0,
               "Low":0.25,"Normal":0.5,"Med":0.75,"Medium":0.75,"High":1.0,
               "OFF":0,"ON":1,"Off":0,"On":1, False:0, True:1}
    if s.dtype == object:
        s = s.map(lambda x: cat_map.get(x, x))
    s = pd.to_numeric(s, errors="coerce").dropna()
    if s.empty:
        return None, None, None, None
    tail = s.tail(96)  # 24 h @ 15 min
    return float(s.iloc[-1]), float(tail.min()), float(tail.max()), float(tail.mean())

# ---------- main ----------
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--ttl", default="~/DTE/jne_project/semantic/2025-03/training_room_semantic_full.ttl")
    ap.add_argument("--csv", default="~/DTE/jne_project/model/2025-03/training_room_model_dataset.csv")
    ap.add_argument("--out", default="~/DTE/jne_project/semantic/2025-03/training_room_graph_readable")  # sans extension
    args,_ = ap.parse_known_args()

    ttl_path = Path(args.ttl).expanduser()
    csv_path = Path(args.csv).expanduser()
    out_base = Path(args.out).expanduser()
    out_base.parent.mkdir(parents=True, exist_ok=True)

    g = rdflib.Graph().parse(str(ttl_path), format="turtle")
    df = pd.read_csv(csv_path)

    # ----- Graphviz compact A4 paysage -----
    G = pgv.AGraph(
        directed=True, strict=False,
        rankdir="LR", splines="spline",
        overlap="false", concentrate="true",
        nodesep="0.45", ranksep="0.7"
    )
    G.graph_attr.update(
        fontname="DejaVu Sans", fontsize="12",
        outputorder="edgesfirst",
        size="20,12!", ratio="compress",
        center="true", margin="0.1", pad="0.1",
        bgcolor="white", dpi="300"   # <- résolution PNG
    )

    STYLES = {
        "hier":   dict(shape="box",     style="filled,rounded", fillcolor="#b3cde0"),
        "room":   dict(shape="ellipse", style="filled",         fillcolor="#ccebc5"),
        "env":    dict(shape="box",     style="filled,rounded", fillcolor="#decbe4"),
        "prop":   dict(shape="ellipse", style="filled",         fillcolor="#fed9a6"),
        "system": dict(shape="box",     style="filled,rounded", fillcolor="#fff2a8"),
        "equip":  dict(shape="box",     style="filled,rounded", fillcolor="#aed6f1"),
        "sensor": dict(shape="ellipse", style="filled",         fillcolor="#f7b7b2"),
        "setpt":  dict(shape="note",    style="filled",         fillcolor="#d9d9d9"),
        "obs":    dict(shape="ellipse", style="filled",         fillcolor="#e0e0e0"),
        "ts":     dict(shape="box",     style="filled,rounded", fillcolor="#ffe4c4"),
        "virt":   dict(shape="ellipse", style="filled",         fillcolor="#d0f0c0"),
    }

    def N(name, label, key):
        G.add_node(name, label=label, fontname="DejaVu Sans", **STYLES[key])
    def E(a,b,label,color="#666666"):
        G.add_edge(a,b, label=label, fontsize="10", color=color, fontcolor=color)

    # clusters
    c_build = G.add_subgraph(name="cluster_build", label="Building hierarchy", color="#7fb3d5")
    c_env   = G.add_subgraph(name="cluster_env",   label="Envelope",           color="#bb8fce")
    c_sys   = G.add_subgraph(name="cluster_sys",   label="Systems",            color="#f7dc6f")
    c_eq    = G.add_subgraph(name="cluster_eq",    label="Equipment",          color="#85c1e9")
    c_sens  = G.add_subgraph(name="cluster_sens",  label="Sensors",            color="#f5b7b1")
    c_obs   = G.add_subgraph(name="cluster_obs",   label="Observations",       color="#d5dbdb")
    c_ts    = G.add_subgraph(name="cluster_ts",    label="Time Series",        color="#f8c471")
    c_leg   = G.add_subgraph(name="cluster_leg",   label="Legend",             color="#bfc9ca")

    for lbl,key in [("Building hierarchy","hier"),("Training Room","room"),("Envelope","env"),
                    ("Properties","prop"),("Systems","system"),("Equipment","equip"),
                    ("Sensors","sensor"),("Setpoints","setpt"),("Observations","obs"),
                    ("Time Series","ts"),("Virtual Weather Sensors","virt")]:
        c_leg.add_node(f"leg_{key}", label=lbl, **STYLES[key])

    # bâtiment
    N("Site", "Site: ENSMR Rabat", "hier");     c_build.add_node("Site")
    N("Building", "Building", "hier");          c_build.add_node("Building")
    N("Floor", "Floor: Ground", "hier");        c_build.add_node("Floor")
    N("Room", "Training Room", "room");         c_build.add_node("Room")
    E("Site","Building","hasPart"); E("Building","Floor","hasPart"); E("Floor","Room","hasPart")

    # enveloppe
    for w in g.subjects(None, EX.Wall):
        wid = short(w)
        N(wid, clean_label(wid), "env"); c_env.add_node(wid); E("Room", wid, "hasPart")
        for p in [EX.r_value_m2K_W, EX.u_value_W_m2K, EX.thickness_m, EX.orientation]:
            for _,_,o in g.triples((w,p,None)):
                pid = f"{wid}_{short(p)}"
                N(pid, f"{short(p)}: {o}", "prop"); c_obs.add_node(pid); E(wid, pid, "hasProperty","#999999")

    # systèmes
    systems = [(EX["HVAC_System"],"HVAC System"),
               (EX["Lighting_System"],"Lighting System"),
               (EX["Weather_Station_1"],"Weather Station")]
    for s_uri, label in systems:
        sid = short(s_uri); N(sid, label, "system"); c_sys.add_node(sid); E("Building", sid, "hasPart")

    # équipements
    for e in g.subjects(None, BRICK.Equipment):
        eid = short(e); N(eid, clean_label(eid), "equip"); c_eq.add_node(eid)
        for sys_uri,_ in systems:
            if (sys_uri, BRICK.hasPart, e) in g: E(short(sys_uri), eid, "hasPart")
        E(eid, "Room", "hasLocation")

    # capteurs TTL + dataset
    for s,_,_ in g.triples((None, BF.hasTimeseriesId, None)):
        sid = short(s); N(sid, clean_label(sid), "sensor"); c_sens.add_node(sid)
        for sys_uri,_ in systems:
            if (sys_uri, BRICK.hasPoint, s) in g or (sys_uri, BRICK.hasPart, s) in g:
                E(short(sys_uri), sid, "hasPoint")
        E(sid, "Room", "hasLocation","#999999")

        if sid in df.columns:
            val, vmin, vmax, vmean = last_and_stats(df, sid)
            oid = f"obs_{sid}"; N(oid, "Observation", "obs"); c_obs.add_node(oid); E(sid, oid, "hasObservation","#888888")
            vtext = f"{val:.2f}" if isinstance(val, (int,float)) and val is not None else str(df[sid].iloc[-1])
            vid = f"val_{sid}"; N(vid, vtext, "prop"); c_obs.add_node(vid); E(oid, vid, "hasSimpleResult")
            tid = f"ts_{sid}"
            tslab = f"TS (24h)\\nMin: {vmin:.2f}\\nMax: {vmax:.2f}\\nMean: {vmean:.2f}" if None not in (vmin,vmax,vmean) else "TS (24h) — n/a"
            G.add_node(tid, label=tslab, fontname="DejaVu Sans", **STYLES["ts"]); c_ts.add_node(tid); E(oid, tid, "hasTimeSeries","#999999")

    # météo virtuelle depuis dataset
    weather_cols = {
        "Weather_Temp_Sensor":"Outdoor Temperature Sensor",
        "Weather_RH_Sensor":"Outdoor Humidity Sensor",
        "Weather_GHI_Sensor":"Solar Irradiance Sensor",
        "Weather_Wind_Sensor":"Wind Speed Sensor",
    }
    for col,label in weather_cols.items():
        if col in df.columns:
            nid = f"virt_{col}"; N(nid, label, "virt"); c_sens.add_node(nid); E("Site", nid, "hasSystem")
            val, vmin, vmax, vmean = last_and_stats(df, col)
            oid = f"obs_{nid}"; N(oid, "Observation", "obs"); c_obs.add_node(oid); E(nid, oid, "hasObservation")
            vtext = f"{val:.2f}" if isinstance(val,(int,float)) and val is not None else "n/a"
            vid = f"val_{nid}"; N(vid, vtext, "prop"); c_obs.add_node(vid); E(oid, vid, "hasSimpleResult")
            tid = f"ts_{nid}"
            tslab = f"TS (24h)\\nMin: {vmin:.2f}\\nMax: {vmax:.2f}\\nMean: {vmean:.2f}" if None not in (vmin,vmax,vmean) else "TS (24h) — n/a"
            G.add_node(tid, label=tslab, fontname="DejaVu Sans", **STYLES["ts"]); c_ts.add_node(tid); E(oid, tid, "hasTimeSeries")

    # rendu
    G.layout(prog="dot")
    svg_path = out_base.with_suffix(".svg")
    png_path = out_base.with_suffix(".png")
    G.draw(str(svg_path), format="svg")
    G.draw(str(png_path), format="png")  # dpi déjà défini dans graph_attr
    print("OK →", svg_path)
    print("OK →", png_path)

if __name__ == "__main__":
    main()


OK → /home/amina/DTE/jne_project/semantic/2025-03/training_room_graph_readable.svg
OK → /home/amina/DTE/jne_project/semantic/2025-03/training_room_graph_readable.png
